<a href="https://colab.research.google.com/github/devrihartle/DS-Unit-2-Kaggle-Challenge/blob/master/Copy_of_DS_module_project_221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [19]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [20]:
import pandas as pd

df = pd.merge(pd.read_csv(DATA_PATH + 'waterpumps/train_features.csv'),
              pd.read_csv(DATA_PATH + 'waterpumps/train_labels.csv'))
# YOUR CODE HERE
df.set_index('id', inplace=True)
#raise NotImplementedError()

df.head()


amount_tsh date_recorded  ... waterpoint_type_group    status_group
id                               ...                                      
69572      6000.0    2011-03-14  ...    communal standpipe      functional
8776          0.0    2013-03-06  ...    communal standpipe      functional
34310        25.0    2013-02-25  ...    communal standpipe      functional
67743         0.0    2013-01-28  ...    communal standpipe  non functional
19728         0.0    2011-07-13  ...    communal standpipe      functional

[5 rows x 40 columns]

**Test 2**

In [21]:
'''Task 2 Testing'''

assert isinstance(df, pd.DataFrame), 'Have you created the DataFrame `df`?'
assert df.shape == (59400, 40), '`df` is the wrong shape. Did you set the index to `id`?'
assert 69572 in df.index, 'The index for `df` is has the wrong values. Did you set the index to `id`?'

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [22]:
target = 'status_group'

X = df.drop(['status_group'], axis=1)
y = df[target]

# YOUR CODE HERE
#raise NotImplementedError()
X.shape

(59400, 39)

**Test 3**

In [23]:
'''Task 3 Testing'''

assert isinstance(X, pd.DataFrame), '`X` is the wrong data type.'
assert isinstance(y, pd.Series), '`y` is the wrong data type.'
assert y.shape == (59400,), '`y` is the wrong shape.'
assert X.shape == (59400,39), '`X` is the wrong shape.'

We already have a test set for this model, the `test.csv` that you'll use to make the predictions you'll upload to Kaggle. However, since our competition only allows for 2 submissions per day, we need a way to estimate our training error so that we can quickly iterate and improve our model. We can achieve this by creating a validation set from the data we have.

**Task 4:** Split `X` and `y` into training and validation sets. Your validation set should be 20% of the data you have. You should have four variables: `X_train`, `X_val`, `y_train`, and `y_val`. 

In [24]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=.80, test_size=.20, stratify=df['status_group'], random_state=42)
#raise NotImplementedError()
#X_train = train[X]
#X_val = val[X]
#y_train = train[target]
#y_val = val[target]
#y_train.shape, y_val.shape
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 39), (11880, 39), (47520,), (11880,))

**Test 4**

In [25]:
'''Task 4 Testing'''

assert X_train.shape == (47520, 39), '`X_train` is the wrong shape.'
assert X_val.shape == (11880, 39), '`X_val` is the wrong shape.'
assert y_train.shape == (47520,), '`y_train` is the wrong shape.'
assert y_val.shape == (11880,), '`y_val` is the wrong shape.'

## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [26]:
# YOUR CODE HERE
y_train.value_counts(normalize=True)
majority_class = y_train.mode()[0]
y_pred = [majority_class]*len(y_train)
from sklearn.metrics import accuracy_score
baseline_acc = accuracy_score(y_train, y_pred)
#raise NotImplementedError()

print('Baseline Accuracy:', baseline_acc)

Baseline Accuracy: 0.5430765993265994


**Task 5**

In [27]:
'''Task 5 Testing'''

assert isinstance(baseline_acc, float), '`baseline_acc` should be a `float`.'
assert 0.0 <= baseline_acc <= 1.0, '`baseline_acc` is a score that should be between 0 and 1.'

## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [31]:
def wrangle(X):
  X = X.copy()

  X.drop(columns='recorded_by', inplace=True)

  X.drop(columns=['quantity_group', 'extraction_type_group'], inplace=True)

  hc_cols = [col for col in X.select_dtypes('object').columns
             if X[col].nunique() > 100]
  X.drop(columns=hc_cols, inplace=True)

  #X['yr_recorded'] = X['date_recorded'].dt.year
  #X.drop(columns='date_recorded', inplace=True)

  return X

In [32]:
train = wrangle(df)

In [33]:
target = 'status_group'
y=df[target]
X = df.drop(columns=target)

In [34]:
from sklearn.model_selection import train_test_split 
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=.80, test_size=.20, stratify=df['status_group'], random_state=42)

In [35]:
df.head()

amount_tsh date_recorded  ... waterpoint_type_group    status_group
id                               ...                                      
69572      6000.0    2011-03-14  ...    communal standpipe      functional
8776          0.0    2013-03-06  ...    communal standpipe      functional
34310        25.0    2013-02-25  ...    communal standpipe      functional
67743         0.0    2013-01-28  ...    communal standpipe  non functional
19728         0.0    2011-07-13  ...    communal standpipe      functional

[5 rows x 40 columns]

In [36]:
df.drop(columns='date_recorded', inplace=True)

In [37]:
df.head()

amount_tsh        funder  ...  waterpoint_type_group    status_group
id                               ...                                       
69572      6000.0         Roman  ...     communal standpipe      functional
8776          0.0       Grumeti  ...     communal standpipe      functional
34310        25.0  Lottery Club  ...     communal standpipe      functional
67743         0.0        Unicef  ...     communal standpipe  non functional
19728         0.0   Action In A  ...     communal standpipe      functional

[5 rows x 39 columns]

In [44]:

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# YOUR CODE HERE

model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'), 
    DecisionTreeClassifier(random_state=42)
)

#print('Train Accuracy', model.score(X_train, y_train))
#print('Validation Accuracy', model.score(X_val, y_val))

#raise NotImplementedError()

model.fit(X_train, y_train);
#print('Train Accuracy', model.score(X_train, y_train))
#print('Validation Accuracy', model.score(X_val, y_val))

**Test 6**

In [45]:
'''Task 6 Testing'''

assert len(model) > 1, 'Your model pipeline should have multiple steps.'
assert isinstance(model[-1], DecisionTreeClassifier), 'Your pipline should end in a `DecisionTreeClassifier`.'
assert hasattr(model, 'classes_'), 'Have you fit your model?'

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [47]:
# YOUR CODE HERE
#raise NotImplementedError()
training_acc = model.score(X_train, y_train)
validation_acc = model.score(X_val, y_val)
print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.9999789562289563
Validation Accuracy: 0.7015151515151515


**Test 7**

In [48]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [56]:
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id')
# YOUR CODE HERE
#raise NotImplementedError()
y_pred = model.predict(X_test)
print(len(X_test))
print(len(y_pred))
#submission.head()
X_test.index

14358
14358


Int64Index([50785, 51630, 17168, 45559, 49871, 52449, 24806, 28965, 36301,
            54122,
            ...
            59757, 64579, 57731, 65541, 68174, 39307, 18990, 28749, 33492,
            68707],
           dtype='int64', name='id', length=14358)

In [59]:
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
sample_submission.head()

id status_group
0  50785   functional
1  51630   functional
2  17168   functional
3  45559   functional
4  49871   functional

In [61]:
submission=pd.DataFrame(y_pred, columns=['status_group'])
submission['id']=sample_submission.id
submission=submission[['id', 'status_group']]
submission.head()

id             status_group
0  50785               functional
1  51630  functional needs repair
2  17168               functional
3  45559           non functional
4  49871               functional

**Test 8**

In [62]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [64]:
# YOUR CODE HERE
submission.to_csv('submission_file.csv',index=False)
#raise NotImplementedError()

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE HERE
raise NotImplementedError()